## 머신러닝 Pipeline
- 이전 강의들에서 결측치 처리, 모델 학습 등을 진행하였다. 
- 이 때, 결측치 처리, 스케일링, 하이퍼 파라미터 등을 최소화하여 쉽게 연결할 수 있도록 도와준다. 
- 본 예제에서는 scikit-learn pipeline에 대해 학습할 예정이다. 

## 필수 라이브러리 불러오기
- 본 튜토리얼에 적합한 주요 라이브러리들을 불러온다. 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

print(f"numpy version {np.__version__}")
print(f"pandas version {pd.__version__}")
print(f"seaborn version {sns.__version__}")
print(f"sklearn version {sklearn.__version__}")

numpy version 1.26.1
pandas version 2.1.1
seaborn version 0.13.0
sklearn version 1.3.1


## 데이터 불러오기
- 본 실습을 위해 간단한 데이터를 불러온다. 

In [4]:
# DATA_PATH = "C:\\Users\\human\\Desktop\\mlops_tutorial\\data\\bike-sharing-demand\\"
DATA_PATH = "C:\\Users\\MSYUN\\Desktop\\ml_optuna_mlflow\\data\\bike-sharing-demand\\"

train_df = pd.read_csv(DATA_PATH + "train.csv")
test = pd.read_csv(DATA_PATH + "test.csv")
submission = pd.read_csv(DATA_PATH + "sampleSubmission.csv")

train_df.shape, test.shape, submission.shape

((10886, 12), (6493, 9), (6493, 2))

## 데이터 전처리
- 데이터 전처리는 기존 강의와 비슷하게 진행한다. 
- 단, 범주형 데이터셋은 그대로 놔두도록 한다. 

In [5]:
# 타깃값 별도 저장
y = train_df['count'] # 타깃값

# count 컬럼 제거
train_df = train_df.drop(['count'], axis=1)

# 데이터 합치기
all_df = pd.concat([train_df, test])

# 날짜 데이터로 파생변수 만들기
all_df['date'] = pd.to_datetime(all_df['datetime'])
all_df['year'] = all_df['date'].dt.year
all_df['date'] = pd.to_datetime(all_df['datetime'])
all_df['year'] = all_df['date'].dt.year
all_df['month'] = all_df['date'].dt.month
all_df['day'] = all_df['date'].dt.day
all_df['hour'] = all_df['date'].dt.hour
all_df['weekday'] = all_df['date'].dt.day_name()

# 원-핫 인코딩을 위해 변경함
season_num = [1, 2, 3, 4]
season_str = ['Spring', 'Summer', 'Fall', 'Winter']
all_df['season'] = all_df['season'].replace(season_num, season_str)

weather_num = [1, 2, 3, 4]
weather_str = ['Clear', 'Few Clouds', 'Light Snow, Rain', 'Heavy Snow, Rain']
all_df['weather'] = all_df['weather'].replace(season_num, season_str)

del_features = ['casual', 'registered', 'datetime', 'date', 'windspeed', 'month', 'atemp']
all_df = all_df.drop(del_features, axis=1)

all_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17379 entries, 0 to 6492
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      17379 non-null  object 
 1   holiday     17379 non-null  int64  
 2   workingday  17379 non-null  int64  
 3   weather     17379 non-null  object 
 4   temp        17379 non-null  float64
 5   humidity    17379 non-null  int64  
 6   year        17379 non-null  int32  
 7   day         17379 non-null  int32  
 8   hour        17379 non-null  int32  
 9   weekday     17379 non-null  object 
dtypes: float64(1), int32(3), int64(3), object(3)
memory usage: 1.3+ MB


## 데이터셋 분리

In [6]:
train = all_df.iloc[0:len(y), :]
test = all_df.iloc[len(y):, :]
train.shape, test.shape

((10886, 10), (6493, 10))

In [7]:
# train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.3, shuffle=True, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((7620, 10), (3266, 10), (7620,), (3266,))

## 데이터 전처리 파이프라인 만들기
- (1) 수치형 변수에 대해서는 StandardScaler() 클래스를 이용해서 표준화를 진행했다. 
- (2) 범주형 변수의 원핫인코딩하는 데이터 전처리 파이프라인을 만든다. 

In [32]:
# (1) 수치형 변수 
num_features = ['temp', 'humidity']
num_transformer = Pipeline(
    steps = [('scaler', StandardScaler())]
)

# (2) 범주형 변수, 원핫 인코딩
cat_features = ['season', 'weather', 'weekday']
cat_transformer = OneHotEncoder(handle_unknown='ignore')

# (3) ColumnTransformer
preprocessor = ColumnTransformer(
    transformers = [
        ("num", num_transformer, num_features), 
        ("cat", cat_transformer, cat_features)
    ]
)

# (4) 모델 정의
pipe_lr_model = Pipeline(
    steps = [("preprocessor", preprocessor), ("regressor", LinearRegression())]
)

In [33]:
from sklearn import set_config 
set_config(display = 'diagram')

# 수치 데이터, 로그변환
log_y = np.log(y_train)
pipe_lr_model.fit(X_train, log_y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'humidity']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['season', 'weather',
                                                   'weekday'])])),
                ('regressor', LinearRegression())])

## 평가지표 구현

In [34]:
def rmsle(y, y_,convertExp=True):
    
    # 지수변환
    if convertExp:
        y = np.exp(y),
        y_ = np.exp(y_)
        
    # 로그변환 후 결측값을 0으로 변환 
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    
    # RMSLE 계산
    return np.sqrt(np.mean(calc))

In [35]:
# 코드
preds = pipe_lr_model.predict(X_valid)
log_y_val = np.log(y_valid)

# 선형회귀 값
print('RMSLE 값:', rmsle(log_y_val, preds, True))

RMSLE 값: 1.1999846010628938


In [36]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6493 entries, 0 to 6492
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      6493 non-null   object 
 1   holiday     6493 non-null   int64  
 2   workingday  6493 non-null   int64  
 3   weather     6493 non-null   object 
 4   temp        6493 non-null   float64
 5   humidity    6493 non-null   int64  
 6   year        6493 non-null   int64  
 7   day         6493 non-null   int64  
 8   hour        6493 non-null   int64  
 9   weekday     6493 non-null   object 
dtypes: float64(1), int64(6), object(3)
memory usage: 558.0+ KB


In [37]:
# 코드
preprocessor.fit_transform(test)
final_preds = pipe_lr_model.predict(test)
submission['count'] = np.round(np.exp(final_preds), 0)
submission.head()

,datetime,count
0,2011-01-20 00:00:00,42.0
1,2011-01-20 01:00:00,42.0
2,2011-01-20 02:00:00,42.0
3,2011-01-20 03:00:00,42.0
4,2011-01-20 04:00:00,42.0


In [38]:
submission.to_csv(DATA_PATH + 'submission.csv', index=False)